# CityBikes

Need to add the relevant libraries first


In [2]:
import requests
import os
import pandas as pd

Explore the structure of the API, query the API and understand the data returned.


In [ ]:
# API End points
# http://api.citybik.es/v2/networks
# {
#   "networks": [
#     {
#         "company": "JCDecaux", 
#         "href": "/v2/networks/velib", <--- network API endpoint
#         "location": {
#           "latitude": 48.856612, 
#           "city": "Paris", 
#           "longitude": 2.352233, 
#           "country": "FRA"
#         }, 
#         "name": "Vélib'", 
#         "id": "velib"
#     },
#     {...}
#   ]
# }
# http://api.citybik.es/v2/networks/network_id

# {
#   "network": {
#     "name": "Vélib'", 
#     "stations": [
#       {
#           "name": "00903 - QUAI MAURIAC  / PONT DE BERCY",   /   UTC Zulu timestamp of the last time
#           "timestamp": "2014-04-14T12:10:17.622Z",  <-------/ the station was updated on our systems
#           "longitude": 2.374340554605615,                       
#           "free_bikes": 1,   <-------------------------- Available bikes
#           "latitude": 48.83713368945151, 
#           "empty_slots": 19,  <------------------------- Empty spaces
#           "id": "f5a551a87eec15155d6409fe9d0ff8e2" <---- Unique id for this station
#       },
#       {...}
#     ], 
#     "company": "JCDecaux",           |
#     "href": "/v2/networks/velib",       |        Redundant Information
#     "location": {                    |
#       "latitude": 48.856612,         |    Just so you know where you are
#       "city": "Paris",               |---      with less requests
#       "longitude": 2.352233,         | 
#       "country": "FRA"               |    This can be filtered by passing
#     },                               |      ?fields=stations to the URI
#     "id": "velib"                    |
#   }
# }


Send a request to CityBikes for the city of your choice. 


In [25]:
# Step 1: Choose a city and get the network ID from the /v2/networks endpoint
city_name = 'Barcelona'
networks_url = 'http://api.citybik.es/v2/networks'
response = requests.get(networks_url)
networks_data = response.json()

In [26]:
# Find the network information for the chosen city
network_info = next((network for network in networks_data['networks'] if network['location']['city'] == city_name), None)

if network_info:
    # Step 2: Use the network ID to retrieve information about bike stations in that city
    network_id = network_info['id']
    api_url = f'http://api.citybik.es/v2/networks/{network_id}'

    response = requests.get(api_url)
    data = response.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [27]:
# Step 3: Extract relevant station information
station_data = []
for station in data['network']['stations']:
        station_info = {
            'latitude': station['latitude'],
            'longitude': station['longitude'],
            'num_bikes': station['free_bikes'],
            
        }
        station_data.append(station_info)

Put your parsed results into a DataFrame.

In [28]:
    # Step 4: Create a Pandas DataFrame
    df = pd.DataFrame(station_data)


In [29]:
 # Step 5: Explore and Analyze the Data
    # Display the first few rows of the DataFrame
print(df.head())

    # Basic statistics
print(df.describe())


    latitude  longitude  num_bikes
0  41.425364   2.185207          7
1  41.430000   2.190246         23
2  41.443365   2.190630          9
3  41.377532   2.170732          1
4  41.375116   2.152309         12
         latitude   longitude   num_bikes
count  506.000000  506.000000  506.000000
mean    41.399846    2.168036    9.784585
std      0.021064    0.024661    7.258115
min     41.346775    2.109154    0.000000
25%     41.383994    2.149077    4.000000
50%     41.396429    2.170662    8.000000
75%     41.412062    2.187232   14.000000
max     41.462095    2.220691   33.000000
